In [ ]:
import cvxpy
from matplotlib import pyplot as plt
import numpy as np
from src.components.microgrid_simple import SimpleMicrogrid
from src.components.battery import Battery, BatteryParameters
from src.utils.tools import set_all_seeds

### Set env vars

In [ ]:
set_all_seeds(0)

# Same env obs as test simple env
mg = SimpleMicrogrid("c_a2c")
demand = mg.demand
pv = mg.pv_gen
price_s = mg.price
emission = mg.emission
price_b = price_s / 4

In [ ]:
battery_params = {
    "soc_0": 0.1,
    "soc_max":0.9,
    "soc_min":0.1,
    "p_charge_max":0.8,
    "p_discharge_max":0.8,
    "efficiency":0.9,
    "capacity":1,
    "sell_price":0.0,
    "buy_price":0.0
    
}

real_battery = Battery(random_soc_0=False, params = BatteryParameters(battery_params))

p_charge, p_discharge, _ = real_battery.check_battery_constraints(power_rate=0.8)
real_battery.apply_action(p_charge=p_charge, p_discharge=p_discharge)

real_battery.soc.item()

# battery.reset()

### Adv battery calc (not working)

In [ ]:
set_all_seeds(0)
battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)

constraints = []
# Battery
    # Starts in 0.1
constraints.append(battery[0] == battery_params["soc_min"])
    # Max and min batteries
for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't
# for i in range(n):
#     constraints.append(action[i] <= 0.9)
#     constraints.append(action[i] >= -0.9)

# Transition
obj = 0

for i in range(n):
    # Max and min battery charge 
    constraints.append(action[i] <= battery_params["p_charge_max"])
    constraints.append(action[i] <= battery_params["p_discharge_max"])
    # Update battery SOC
    # self.soc = self.soc + (p_charge * self.efficiency - p_discharge / self.efficiency) / self.capacity

    constraints.append(battery[i+1] == battery[i] + ((action[i] * battery_params["efficiency"])/battery_params["efficiency"])/battery_params["capacity"] )
    # Update net 
    constraints.append(consumption[i] == demand[i]-pv[i] + action[i] * battery_params["efficiency"])


    obj += cvxpy.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cvxpy.maximum(-consumption[i] * price_b[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value, consumption.value, price_s

### Real dataset

In [ ]:
battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)

constraints = []
# Battery
    # Starts in 0.1
constraints.append(battery[0] == battery_params["soc_min"])
    # Max and min batteries
for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't

for i in range(n):
    constraints.append(action[i] <= 1)
    constraints.append(action[i] >= -1)


# Transition
obj = 0

for i in range(n):
    
    constraints.append(action[i] <= battery_params["p_charge_max"])
    constraints.append(action[i] <= battery_params["p_discharge_max"])
    # Update battery
    constraints.append(battery[i+1] == battery[i] + action[i] * battery_params["efficiency"])
    # Update net 
    constraints.append(consumption[i] == demand[i]-pv[i] + action[i] * battery_params["efficiency"])


    obj += cvxpy.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cvxpy.maximum(-consumption[i] * price_b[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value

### Test Dataset

In [ ]:
n = 4
d = np.random.rand(n)
g = np.random.rand(n)*0.5
pb = np.random.rand(n)
ps = pb/4

In [ ]:

battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)


constraints = []
for i in range(n+1):
    constraints.append(battery[i] <= 1)
    constraints.append(battery[i] >= 0)
constraints.append(battery[0] == 0)

for i in range(n):
    constraints.append(action[i] <= .3)
    constraints.append(action[i] >= -1)
    

obj = 0
for i in range(n):
    constraints.append(battery[i+1] == battery[i] + action[i]  )

    constraints.append(consumption[i] == d[i]-g[i] + action[i] )


    obj += cvxpy.maximum(consumption[i]* pb[i],0) 
    obj += cvxpy.maximum(-consumption[i]*ps[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value

In [ ]:


x_hat = cvxpy.Variable(A.shape[1])
objective = cvxpy.Minimize(cvxpy.norm(x_hat, 1))
    constraints = [A * x_hat == y]
    prob = cvxpy.Problem(objective, constraints)
    prob.solve()
    return np.squeeze(np.array(x_hat.value)), (prob.value, prob.status)